In [1]:
import os
import nltk
import json
import pickle
import re
import string
import pandas as pd
import numpy as np

In [2]:
#nltk.download()

In [3]:
features=['title', 'id', 'release_date', 'lyrics']

In [4]:
df=pd.DataFrame()
for dirname, _, filenames in os.walk('data\json files'):
    for filename in filenames:
        with open(os.path.join(dirname, filename)) as f:
            aux = json.load(f)
            df_tmp=pd.DataFrame()
            Artist= np.full((len(aux['songs'])), aux['name'])
            df_tmp['artist']=Artist
            for f in features:
                col=[song[f] for song in aux['songs']]
                df_tmp[f]=col
            df=pd.concat([df,df_tmp]).reset_index(drop=True)

In [5]:
df

,artist,title,id,release_date,lyrics
0,Ariana Grande,"​thank u, next",4063065,2018-11-03,[Verse 1]\nThought I'd end up with Sean\nBut h...
1,Ariana Grande,7 rings,4067762,2019-01-18,"[Verse 1]\nYeah, breakfast at Tiffany's and bo..."
2,Ariana Grande,​God is a woman,3681280,2018-07-13,"[Chorus]\nYou, you love it how I move you\nYou..."
3,Ariana Grande,Side To Side,2457495,2016-05-20,[Intro: Ariana Grande & Nicki Minaj]\nI've bee...
4,Ariana Grande,​​no tears left to cry,3649172,2018-04-20,"[Intro]\nRight now, I'm in a state of mind\nI ..."
...,...,...,...,...,...
5853,Taylor Swift,Welcome Back Grunwald,6226864,None,Turn WYCD on\nYou're\nOn your Grunwald\nBack f...
5854,Taylor Swift,Tolerate it (Polskie Tłumaczenie),6315848,2020-12-11,[Zwrotka 1]\nSiedzę i patrzę jak czytasz z gło...
5855,Taylor Swift,Find you,6209316,None,Trying just like they say\nJust taking the ste...
5856,Taylor Swift,Static,6371365,None,*static for 8 seconds*


In [39]:
tmp=[]
#tags=['remix','Remix', 'REMIX', 'live', 'Live', 'Acoustic',' Version', ' ver', 'version', 'Demo', 'demo' , 'Reprise', 'Copy', 'Mix','Edit']
tags=['remix', 'live','club', 'acoustic',' version', ' ver', 'demo', 'reprise', 'copy', 'mix','edit']
#tags=['club']
for title in df['title']:
    for tag in tags:
        r=re.search(tag,title.lower())
        #r=title.find(tag)
        if r != None:
            tmp.append(title)

In [40]:
tmp

['In da Club (Sexy Lil Thug)',
 'Club Paradise',
 'Eminem Freestyle On The Farmclub',
 'Meet Me At The Club*',
 'Hatin’ on the Club',
 'You da One (Almighty Club)',
 'Only Girl (In the World) [Extended Club]',
 'S&M (Sidney Samson Club)',
 'Take a Bow (Subkulcha Club)',
 'Take a Bow (Seamus Haji & Paul Emanuel Club)',
 'California King Bed (Bassjackers Club)',
 'California King Bed (DJ Chus & Abel Ramos Club)',
 'You da One (Gregor Salto Vegas Club)',
 'Take a Bow (Tony Moran & Warren Riggs Encore Club)',
 'California King Bed (The Bimbo Jones Club)',
 'We Found Love (Cahill Club)',
 'You da One (Dave Audé Club)',
 'ROCKSTAR 101 (Dave Audé Club)',
 'Take a Bow (Groove Junkies MoHo Club)',
 'S&M (Dave Audé Club)',
 'You da One (Gregor Salto Amsterdam Club)',
 'Pon De Replay - Pon De Club Play',
 'S&M (Joe Bermudez Chico Club)']

In [41]:
for title in tmp:
    df= df[df.title != title].reset_index(drop=True)

In [42]:
df

,artist,title,id,release_date,lyrics
0,Ariana Grande,"​thank u, next",4063065,2018-11-03,[Verse 1]\nThought I'd end up with Sean\nBut h...
1,Ariana Grande,7 rings,4067762,2019-01-18,"[Verse 1]\nYeah, breakfast at Tiffany's and bo..."
2,Ariana Grande,​God is a woman,3681280,2018-07-13,"[Chorus]\nYou, you love it how I move you\nYou..."
3,Ariana Grande,Side To Side,2457495,2016-05-20,[Intro: Ariana Grande & Nicki Minaj]\nI've bee...
4,Ariana Grande,​​no tears left to cry,3649172,2018-04-20,"[Intro]\nRight now, I'm in a state of mind\nI ..."
...,...,...,...,...,...
3828,Taylor Swift,"Taylor Swift - no body, no crime (Traducción a...",6274728,2020-12-11,[Intro: HAIM]\nÉl lo hizo\nÉl lo hizo\n\n[Vers...
3829,Taylor Swift,Welcome Back Grunwald,6226864,None,Turn WYCD on\nYou're\nOn your Grunwald\nBack f...
3830,Taylor Swift,Tolerate it (Polskie Tłumaczenie),6315848,2020-12-11,[Zwrotka 1]\nSiedzę i patrzę jak czytasz z gło...
3831,Taylor Swift,Find you,6209316,None,Trying just like they say\nJust taking the ste...


In [10]:
for f in features:
    print("number null value of {}: ".format(f),sum(df[f].isnull()))

number null value of title:  0
number null value of id:  0
number null value of release_date:  1369
number null value of lyrics:  7


In [11]:
df.dropna(subset = ["lyrics"], inplace=True)
tmp=[]
for idx,lyric in zip(df.id, df.lyrics):
    if (len(lyric.split("\n"))<6 and re.search('\[(.*?)\]', lyric)==None) or len(lyric)<100: 
        tmp.append(idx)
for i in tmp:
    df=df[df.id!=i]
df=df.reset_index(drop=True)

In [12]:
df

,artist,title,id,release_date,lyrics
0,Ariana Grande,"​thank u, next",4063065,2018-11-03,[Verse 1]\nThought I'd end up with Sean\nBut h...
1,Ariana Grande,7 rings,4067762,2019-01-18,"[Verse 1]\nYeah, breakfast at Tiffany's and bo..."
2,Ariana Grande,​God is a woman,3681280,2018-07-13,"[Chorus]\nYou, you love it how I move you\nYou..."
3,Ariana Grande,Side To Side,2457495,2016-05-20,[Intro: Ariana Grande & Nicki Minaj]\nI've bee...
4,Ariana Grande,​​no tears left to cry,3649172,2018-04-20,"[Intro]\nRight now, I'm in a state of mind\nI ..."
...,...,...,...,...,...
3851,Taylor Swift,"Taylor Swift - no body, no crime (Traducción a...",6274728,2020-12-11,[Intro: HAIM]\nÉl lo hizo\nÉl lo hizo\n\n[Vers...
3852,Taylor Swift,Welcome Back Grunwald,6226864,None,Turn WYCD on\nYou're\nOn your Grunwald\nBack f...
3853,Taylor Swift,Tolerate it (Polskie Tłumaczenie),6315848,2020-12-11,[Zwrotka 1]\nSiedzę i patrzę jak czytasz z gło...
3854,Taylor Swift,Find you,6209316,None,Trying just like they say\nJust taking the ste...


# Preprocessing

In [13]:
import preprocess
clean= preprocess.Preprocessing()

In [14]:
lyrics= np.array([clean.Preprocess(lyric, handle_negation=True) for lyric in df.lyrics])

C:\Users\hoang\AppData\Local\Temp\ipykernel_25768\3739097775.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  lyrics= np.array([clean.Preprocess(lyric, handle_negation=True) for lyric in df.lyrics])


In [16]:
df.lyrics[0]

'[Verse 1]\nThought I\'d end up with Sean\nBut he wasn\'t a match\nWrote some songs about Ricky\nNow I listen and laugh\nEven almost got married\nAnd for Pete, I\'m so thankful\nWish I could say, "Thank you" to Malcolm\n\'Cause he was an angel\n\n[Pre-Chorus]\nOne taught me love\nOne taught me patience\nAnd one taught me pain\nNow, I\'m so amazing\nSay I\'ve loved and I\'ve lost\nBut that\'s not what I see\nSo, look what I got\nLook what you taught me\nAnd for that, I say\n\n[Chorus]\nThank you, next (Next)\nThank you, next (Next)\nThank you, next\nI\'m so fuckin\' grateful for my ex\nThank you, next (Next)\nThank you, next (Next)\nThank you, next (Next)\nI\'m so fuckin\'—\n\n[Verse 2]\nSpend more time with my friends\nI ain\'t worried \'bout nothin\'\nPlus, I met someone else\nWe havin\' better discussions\nI know they say I move on too fast\nBut this one gon\' last\n\'Cause her name is Ari\nAnd I\'m so good with that (So good with that)\n\n[Pre-Chorus]\nShe taught me love (Love)\nShe

In [17]:
lyrics

array([list(['thought', 'would', 'end', 'sean', 'disagree', 'wrote', 'songs', 'ricky', 'listen', 'laugh', 'even', 'almost', 'got', 'married', 'pete', 'thankful', 'wish', 'could', 'say', 'thank', 'malcolm', 'cause', 'angel', 'one', 'taught', 'love', 'one', 'taught', 'patience', 'one', 'taught', 'pain', 'amazing', 'say', 'loved', 'lost', 'not', 'see', 'look', 'got', 'look', 'taught', 'say', 'thank', 'next', 'next', 'thank', 'next', 'next', 'thank', 'next', 'fuckin', 'grateful', 'ex', 'thank', 'next', 'next', 'thank', 'next', 'next', 'thank', 'next', 'next', 'fuckin', 'spend', 'time', 'friends', 'reassure', 'bout', 'nothin', 'plus', 'met', 'someone', 'else', 'havin', 'better', 'discussions', 'know', 'say', 'move', 'fast', 'one', 'gon', 'last', 'cause', 'name', 'ari', 'good', 'good', 'taught', 'love', 'love', 'taught', 'patience', 'patience', 'handles', 'pain', 'pain', 'shits', 'amazing', 'yeah', 'amazing', 'loved', 'lost', 'yeah', 'yeah', 'not', 'see', 'yeah', 'yeah', 'cause', 'look', 'fo

In [23]:
id= np.array([id for id in df.id])

In [18]:
tokenize_lyric=[]
from tqdm import tqdm
for lyric in tqdm(lyrics):
    tokenize_lyric.append(lyric)


100%|█████████████████████████████████████████████████████████████████████████| 3856/3856 [00:00<00:00, 3873829.04it/s]


In [24]:
for i in range(len(df)):
    tokenize_lyric[i].append(f"-->{id[i]}")

In [25]:
from rank_bm25 import BM25Okapi, BM25Plus

In [26]:
bm25 = BM25Plus(tokenize_lyric)

In [27]:
def bm25okapi_search(tokenized_query, bm25, corpus, n_results = 1):
    """
    Function that takes a tokenized query and prints the first 100 words of the 
    n_results most relevant results found in the corpus, based on the BM25
    method.
    
    Parameters
    ----------
    @param tokenized_query: list, array-like
        A valid list containing the tokenized query.
    @param bm25: BM25 object,
        A valid object of type BM25 (BM25Okapi or BM25Plus) from the library
        `rank-bm25`, initialized with a valid corpus.
    @param corpus: list, array-like
        A valid list containing the corpus from which the BM25 object has been 
        initialized. As returned from function read_corpus().
    @param n_results: int, default = 1
        The number of top results to print.
    """
    
    # We skip checking validity of arguments for now... We assume the user 
    # knows what they're doing.
    
    # Get top results for the query
    top_results = bm25.get_top_n(tokenized_query, corpus, n = n_results)
    
    # Take words from each result
    top_results_id = [' '.join(top_result).split("-->")[-1] 
                             for top_result in top_results]
    
    # Return results
    return top_results_id

In [28]:
corpus= "Sorry"

In [29]:
corpus=clean.Preprocess(corpus)
corpus=[corpus.split(" ")]


In [30]:
corpus

[['sorry']]

In [31]:
rank=bm25okapi_search(corpus[0], bm25, tokenize_lyric, n_results= 20)

In [32]:
for i in rank:
    print(df[df.id==int(i)].title)

1989    Sorry
Name: title, dtype: object
409    ​listen before i go
Name: title, dtype: object
186    Sorry
Name: title, dtype: object
2113    All I Want Is You
Name: title, dtype: object
3625    Breathe
Name: title, dtype: object
2286    It Takes Two
Name: title, dtype: object
1263    Genesis
Name: title, dtype: object
3707    You Don’t Have To Call  Me
Name: title, dtype: object
3326    Take a Bow (Tony Moran & Warren Riggs)
Name: title, dtype: object
3367    Take a Bow (Tony Moran & Warren Riggs Encore C...
Name: title, dtype: object
3338    Take a Bow (Subkulcha Club)
Name: title, dtype: object
10    ​needy
Name: title, dtype: object
3378    Take a Bow (Groove Junkies MoHo Club)
Name: title, dtype: object
3345    Take a Bow (Seamus Haji & Paul Emanuel Club)
Name: title, dtype: object
2741    The Air That I Breathe (iTunes Exclusive)
Name: title, dtype: object
3200    Take a Bow
Name: title, dtype: object
829    Goodbye and Goodnight
Name: title, dtype: object
1914    Eminem, Paul R

In [33]:
query= "we don't talk anymore"

In [ ]:
import nltk
from nltk.corpus import wordnet

In [ ]:
print(negation_handler(nltk.word_tokenize(query)))

In [ ]:
tokens= nltk.word_tokenize(query)

In [ ]:
tokens

In [ ]:
wordnet.synsets(tokens[3])[0].lemmas()[1].antonyms()

In [ ]:
def negation_handler(sentence):
    temp = int(0)
    for i in range(len(sentence)):
        if sentence[i-1] in ['not',"n't"]:
            antonyms = []
            for syn in wordnet.synsets(sentence[i]):
                syns = wordnet.synsets(sentence[i])
                w1 = syns[0].name()
                temp = 0
                for l in syn.lemmas():
                    if l.antonyms():
                        antonyms.append(l.antonyms()[0].name())
                max_dissimilarity = 0
                for ant in antonyms:
                    syns = wordnet.synsets(ant)
                    w2 = syns[0].name()
                    syns = wordnet.synsets(sentence[i])
                    w1 = syns[0].name()
                    word1 = wordnet.synset(w1)
                    word2 = wordnet.synset(w2)
                    if isinstance(word1.wup_similarity(word2), float) or isinstance(word1.wup_similarity(word2), int):
                        temp = 1 - word1.wup_similarity(word2)
                    if temp>max_dissimilarity:
                        max_dissimilarity = temp
                        antonym_max = ant
                        sentence[i] = antonym_max
                        sentence[i-1] = ''
    while '' in sentence:
        sentence.remove('')
    return sentence

In [ ]:
wordnet.synsets('hello')